# Fusing the images 🌌

This section aims to fuse the previously processed images. By now, all of the images represent the same area.

### Basic imports

In [1]:
#load the modules
import numpy as np
import pandas as pd
from PIL import Image
import os
import math
from itertools import cycle , islice
import glob

### Gram-Schmidt Fusion for Multispectral Image Preprocessing

This part performs preprocessing and fusion of multispectral images using a Gram-Schmidt (GS) like method to generate enhanced pseudo-RGB images. The pipeline consists of the following key components:

- **`preprocess_annons`**: Loads a CSV file containing image annotations into a Pandas DataFrame for potential downstream processing.

- **`gram_schmidt_fusion_rgb`**: Applies the Gram-Schmidt fusion technique to combine multispectral bands into a pseudo-RGB image. The fusion is based on an intensity substitution method and supports an adjustable boost factor (`alpha`) to emphasize multispectral contribution.

- **Main Loop**:
  - Iterates through groups of 5 aligned images: typically one reference JPEG and four multispectral bands (e.g., NIR, Red, Red Edge).
  - Constructs two image stacks:
    - `fused_imgs`: A full multispectral stack.
    - `pseudo_rgb`: A 3-channel RGB approximation using selected bands.
  - Applies GS fusion and saves both the fused pseudo-RGB image and the standard RGB image for comparison.

- **Output**:
  - Fused images are saved in a `fused-imgs` directory.
  - Corresponding RGB reference images are saved in a `rgb-imgs` directory.

This process supports image enhancement and data preparation for machine learning models, especially in remote sensing and vegetation analysis contexts.


In [ ]:
def preprocess_annons(csv_path):
    """
    Preprocess the annotations CSV file to extract relevant information.
    Args:
        csv_path (str): Path to the CSV file containing annotations.
    Returns:
        pd.DataFrame: DataFrame containing the processed annotations.
    """
    # Read the CSV file
    df = pd.read_csv(csv_path)
    return df


def gram_schmidt_fusion_rgb(multispectral, pseudo_rgb, alpha=2):
    """
    Fuse the multispectral information into a pseudo-RGB image using an intensity substitution approach,
    with an adjustable multispectral boost factor.

    Args:
        multispectral (np.ndarray): Array of shape (H, W, C_ms) from all bands.
        pseudo_rgb (np.ndarray): Array of shape (H, W, 3) composed from selected bands (e.g., R, G, B).
        alpha (float): Boost factor to increase the multispectral impact.

    Returns:
        np.ndarray: Fused pseudo-RGB image (H, W, 3) as uint8.
    """
    pseudo_rgb = pseudo_rgb.astype(np.float32)
    multispectral = multispectral.astype(np.float32)

    intensity_rgb = np.mean(pseudo_rgb, axis=-1, keepdims=True)
    intensity_ms = np.mean(multispectral, axis=-1, keepdims=True)
    # boost the multispectral intensity before computing the ratio
    ratio = (alpha * intensity_ms) / (intensity_rgb + 1e-8)  # avoid division by zero

    fused = pseudo_rgb * ratio
    fused = np.clip(fused, 0, 255).astype(np.uint8)
    return fused

# main processing of images for GS fusion
current_imgs_dir = "./preprocessed-imgs"
out_dir = "./fused-imgs"
os.chdir(current_imgs_dir)
types = ('*.JPG', '*.TIF')
files_grabbed = []
for pattern in types:
    files_grabbed.extend(glob.glob(pattern))
    
input_imgs = sorted(files_grabbed)  
i = cycle(input_imgs)
slc = 5  # number of images per group

# process images in groups of 5
for _ in range(math.ceil(len(input_imgs) / slc)):
    cur_imgs = list(islice(i, slc))
    if len(cur_imgs) < slc:
        break
        
    # determine destination folder based on current image's basename.
    base_name = os.path.basename(cur_imgs[0][:-4]) + '_GS.jpg'

    
    # load images from current group
    ref_img = np.asarray(Image.open(cur_imgs[1]))
    target_jpg = np.asarray(Image.open(cur_imgs[0])) * 0.125
    target_nir = np.asarray(Image.open(cur_imgs[2])) 
    target_r = np.asarray(Image.open(cur_imgs[3])) 
    target_re = np.asarray(Image.open(cur_imgs[4])) 
    
    # stack images to form multispectral and pseudo-RGB inputs
    fused_imgs = np.dstack((ref_img, target_jpg, target_nir, target_r, target_re))
    # example pseudo-RGB: using target_r for red, ref_img for green, and target_nir for blue.
    pseudo_rgb = np.dstack((target_r, ref_img, target_nir))
    
    fused_pseudo_rgb = gram_schmidt_fusion_rgb(multispectral=fused_imgs, pseudo_rgb=pseudo_rgb)

    dest_folder_fused = os.path.dirname(os.getcwd()) + "/fused-imgs"
    dest_folder_rgb = os.path.dirname(os.getcwd()) + "/rgb-imgs"

    output_filename = os.path.join(dest_folder_fused, f"{os.path.splitext(base_name)[0]}.jpg")
    output_filename_rgb = os.path.join(dest_folder_rgb, f"{os.path.splitext(base_name)[0][:-5]}_RGB.jpg")

    # save the fused image and rgb image
    Image.fromarray((np.asarray(Image.open(cur_imgs[0]))).astype(np.uint8)).save(output_filename_rgb)
    Image.fromarray(fused_pseudo_rgb).save(output_filename)

    print(f"Saved fused image: {output_filename}")
    print(f"Saved RGB image: {output_filename_rgb}")


print("All images transformed and saved.")


Saved fused image: /mnt/c/Users/eolic/OneDrive/Documentos/GitHub/cerrado-tree-id/preprocessing/fused-imgs/processed-DJI_20250127100947_0154_D_GS.jpg
Saved RGB image: /mnt/c/Users/eolic/OneDrive/Documentos/GitHub/cerrado-tree-id/preprocessing/rgb-imgs/processed-DJI_20250127100947_0154_RGB.jpg
Saved fused image: /mnt/c/Users/eolic/OneDrive/Documentos/GitHub/cerrado-tree-id/preprocessing/fused-imgs/processed-DJI_20250127100949_0155_D_GS.jpg
Saved RGB image: /mnt/c/Users/eolic/OneDrive/Documentos/GitHub/cerrado-tree-id/preprocessing/rgb-imgs/processed-DJI_20250127100949_0155_RGB.jpg
Saved fused image: /mnt/c/Users/eolic/OneDrive/Documentos/GitHub/cerrado-tree-id/preprocessing/fused-imgs/processed-DJI_20250127100952_0156_D_GS.jpg
Saved RGB image: /mnt/c/Users/eolic/OneDrive/Documentos/GitHub/cerrado-tree-id/preprocessing/rgb-imgs/processed-DJI_20250127100952_0156_RGB.jpg
Saved fused image: /mnt/c/Users/eolic/OneDrive/Documentos/GitHub/cerrado-tree-id/preprocessing/fused-imgs/processed-DJI_2